# Item 27: Use Comprehensions Instead of map and filter

In [61]:
a = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

squares = [x**2 for x in a]  #list comprehension
print(squares)

alt = map(lambda x: x**2, a) #map
print(list(alt))

[1, 4, 9, 16, 25, 36, 49, 64, 81, 100]
[1, 4, 9, 16, 25, 36, 49, 64, 81, 100]


In [62]:
even_squares = [x**2 for x in a if x % 2 == 0]
print(even_squares)

even_alt = map(lambda x: x**2, filter(lambda x: x % 2 == 0, a)) #filter까지 써야함
print(list(even_alt))

[4, 16, 36, 64, 100]
[4, 16, 36, 64, 100]


In [63]:
even_squares_dict = {x: x**2 for x in a if x % 2 == 0} #딕셔너리도 만들 수 있음
print(even_squares_dict)

even_squares_set = {x**2 for x in a if x % 2 == 0} #set도 만들 수 있음
print(even_squares_set)

{2: 4, 4: 16, 6: 36, 8: 64, 10: 100}
{64, 100, 4, 36, 16}


# Item 28: Avoid more than Two Control Subexpressions in Comprehensions

In [64]:
a = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
b = [x for x in a if x > 4 if x % 2 == 0] #이런식으로 쓰지 말것
print(b)

[6, 8, 10]


# Item 29: Avoid Repeated Work in Comprehensions by Using Assignment Expressions

In [65]:
stock = {
    'nails' : 125, 
    'screws' : 35,
    'wingnuts' : 8,
    'washers' : 24,
}

order = ['screws', 'wingnuts', 'clips']

def get_batches(count, size):
    return count // size

found = {name: get_batches(stock.get(name, 0), 8) 
         for name in order 
         if get_batches(stock.get(name, 0), 8)} #같은 부분이 반복됨
print(found)

found = {name: batches for name in order
         if (batches := get_batches(stock.get(name, 0),8))} #Assignment Expression
print(found)

{'screws': 4, 'wingnuts': 1}
{'screws': 4, 'wingnuts': 1}


# Item 30: Consider Generation Instead of Returning Lists

In [66]:
def index_words_iter(text): #generator function 
    if text:
        yield 0
    for index, letter in enumerate(text):
        if letter == ' ':
            yield index + 1

address = 'Four score and seven years ago...'

it = index_words_iter(address) #generator
print(next(it))
print(next(it))

0
5


In [67]:
result = list(index_words_iter(address))
print(result)

[0, 5, 11, 15, 21, 27]


# Item 31: Be Defensive When Iterating Over Arguments

# Item 32: Conider Generator Expressions for Large List Comprehensions

In [68]:
it = (len(x) for x in open('my_file.txt','rt', encoding='UTF8')) #generator expression () -> memory 문제 해결
print(it)

<generator object <genexpr> at 0x0000020299BA8890>


In [69]:
print(next(it))
print(next(it))
print(next(it))

34
40
36


# Item 33: Compose Multiple Generators with 'yield from'

In [70]:
def move(period, speed):
    for _ in range(period):
        yield speed

def pause(delay):
    for _ in range(delay):
        yield 0


In [71]:
def animate_composed():#generator function
    yield from move(4, 5.0) #generator1
    yield from pause(3)     #generator2
    yield from move(2, 3.0) #generator3 

def render(delta):
    print(f'Delta: {delta:.1f}')
    
def run(func):
    for delta in func():
        render(delta)

In [72]:
run(animate_composed)

Delta: 5.0
Delta: 5.0
Delta: 5.0
Delta: 5.0
Delta: 0.0
Delta: 0.0
Delta: 0.0
Delta: 3.0
Delta: 3.0


# Item 34: Avoid Injecting Data into Generators with send

In [73]:
def my_generator():
    received = yield 1
    print(f'receivied = {received}')

it = my_generator()
output = it.send(None) #무조건 처음에는 None 객체
print(output)
try:
    it.send('hello!') #이런식으로 send쓰지 마라
except StopIteration:
    pass

1
receivied = hello!


# Item 35: Avoid Causing State Transitions in Generators with throw

In [74]:
class MyError(Exception):
    pass

def my_generator():
    yield 1
    yield 2
    yield 3

it = my_generator()
print(next(it))
print(next(it))
#print(it.throw(MyError('test error'))) #이렇게 사용하지 말것,

1
2


# Item 36: Consider itertools for Working with Iterators and Generators

In [75]:
import itertools

#Linking Iterators Together

#chain
it = itertools.chain([1,2,3], [4,5,6])
print(list(it))

#repeat
it = itertools.repeat('hello', 3)
print(list(it))

#cycle
it = itertools.cycle([1,2])
result = [next(it) for _ in range (10)]
print(result)

#tee
it1, it2, it3 = itertools.tee(['first', 'second'], 3)
print(list(it1))
print(list(it2))
print(list(it3))

#zip_longest
keys = ['one', 'two', 'three']
values = [1, 2]
it = itertools.zip_longest(keys, values, fillvalue='Nope')
print(list(it))

[1, 2, 3, 4, 5, 6]
['hello', 'hello', 'hello']
[1, 2, 1, 2, 1, 2, 1, 2, 1, 2]
['first', 'second']
['first', 'second']
['first', 'second']
[('one', 1), ('two', 2), ('three', 'Nope')]


In [76]:
# Filtering Items from an Iterator

#islice
values = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
middle_odds = itertools.islice(values, 2, 8, 2)
print(list(middle_odds))

#takewhile
values = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
less_than_seven = lambda x: x < 7
it = itertools.takewhile(less_than_seven, values)
print(list(it))

#dropwhile
values = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
less_than_seven = lambda x: x < 7
it = itertools.dropwhile(less_than_seven, values)
print(list(it))

#filterfalse
values = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
evens = lambda x: x % 2 == 0
filter_false_result = itertools.filterfalse(evens, values)
print(list(filter_false_result))

[3, 5, 7]
[1, 2, 3, 4, 5, 6]
[7, 8, 9, 10]
[1, 3, 5, 7, 9]


In [77]:
#Producing Combinations of Items from Iterators

#accumulate
values = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
sum_reduce = itertools.accumulate(values)
print(list(sum_reduce))

#product
single = itertools.product([1, 2], repeat=2)
print(list(single))
multiple = itertools.product([1, 2], ['a', 'b'])
print(list(multiple))

#permutations(순열)
it = itertools.permutations([1, 2, 3, 4], 2)
print(list(it))

#combinations(조합)
it = itertools.combinations([1, 2, 3, 4], 2)
print(list(it))

#combinations_with_replacement(중복 허용한 조합)
it = itertools.combinations_with_replacement([1, 2, 3, 4], 2)
print(list(it))

[1, 3, 6, 10, 15, 21, 28, 36, 45, 55]
[(1, 1), (1, 2), (2, 1), (2, 2)]
[(1, 'a'), (1, 'b'), (2, 'a'), (2, 'b')]
[(1, 2), (1, 3), (1, 4), (2, 1), (2, 3), (2, 4), (3, 1), (3, 2), (3, 4), (4, 1), (4, 2), (4, 3)]
[(1, 2), (1, 3), (1, 4), (2, 3), (2, 4), (3, 4)]
[(1, 1), (1, 2), (1, 3), (1, 4), (2, 2), (2, 3), (2, 4), (3, 3), (3, 4), (4, 4)]
